In [7]:
import cv2
import mediapipe as mp
import numpy as np
from mediapipe.framework.formats import landmark_pb2

mp_face_mesh = mp.solutions.face_mesh

# 학습된 모델 불러오기

In [10]:
from keras.models import load_model

model = load_model('model/model02')

# 코드 리팩토링 및 테스트

In [21]:
land_list = [46, 53, 52, 55, 65, 276, 283, 282, 295, 285, 130, 160, 158, 133, 153, 144, 359, 387, 385, 362, 380, 373,
             168, 6, 195, 4, 61, 39, 0, 269, 291, 405, 17, 181, 234, 132, 152, 288, 454]
cName = "Non-Drowsy"
cap = cv2.VideoCapture(0)
dTime = 0

with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_face_landmarks:
            point_list = []
            for i in land_list:
                point_list.append(results.multi_face_landmarks[0].landmark[i])

            temp = []
            for i, points in enumerate(point_list):
                temp.append(points.x)
                temp.append(points.y)
                temp.append(points.z)
            temp = np.array(temp).reshape(1, 117)
        
            classes = model.predict(temp)[0]
            if(classes[0] < classes[1]):
                if dTime >= 30:
                    cName = "Drowsy !!!"
                else:
                    cName = "Drowsy"
                dTime += 1
            else:
                cName = "Non-Drowsy"
                dTime = 0
            
            image = cv2.flip(image, 1)
            cv2.putText(image, cName + str(dTime), (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (98, 17, 0), 2, cv2.LINE_AA)
            
            cv2.imshow('MediaPipe Face Mesh', image)
            if cv2.waitKey(5) & 0xFF == 27:
                break
                
cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 14ms/step


...

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 16ms/step
